<a href="https://colab.research.google.com/github/Zernach/DS-Unit-2-Linear-Models/blob/master/214_ZERNACH_%E2%80%94_Logistic_Regression_(Thursday%2C_October_31st_2019).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASSIGNMENT COMPLETED BY: [RYAN ZERNACH](http://ryan.zernach.com/portfolio/)

![alt text](http://www.zernach.com/wp-content/uploads/2019/09/Ryan-Zernach-Logo-1-e1568499634499.png)

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [x] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [x] Begin with baselines for classification.
- [x] Use scikit-learn for logistic regression.
- [x] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [x] Get your model's test accuracy. (One time, at the end.)
- [x] Commit your notebook to your fork of the GitHub repo.
- [x] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [x] Do one-hot encoding.
- [x] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

## PART 1 -- Setup the dataset/dataframe.

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/burritos/burritos.csv')

# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

# Make [Date] a datetime object in preparation for PART 2
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

# Cleanup the x's in columns to universalize the data
# (there's even a Yes and a No in the Chips column instead of x's hahah) 
import numpy as np
df['Chips'] = df['Chips'].str.replace('Yes','x')
df['Chips'] = df['Chips'].str.replace('No', 'x') #This is cheating -- 'No' should be replaced with nan but its only one single instance-- how should I handle this?
df = df.drop('Queso', axis=1) #Nothing but nan values

dirty_columns = ('Chips', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
                 'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp',
                 'Fish', 'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper',
                 'Carrots', 'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion',
                 'Taquito', 'Pineapple', 'Ham', 'Chile relleno', 'Nopales',
                 'Lobster', 'Egg', 'Mushroom', 'Bacon', 'Sushi',
                 'Avocado', 'Corn', 'Zucchini')

for column in dirty_columns:
  df[column] = df[column].replace('X','x')

## PART 2 -- Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [0]:
train = df[(df['Date'] >= '01-01-2016') & (df['Date'] <= '12-31-2016')]
validate = df[(df['Date'] >= '01-01-2017') & (df['Date'] <= '12-31-2017')]
test = df[df['Date'] >= '01-01-2018']

## PART 3 -- Begin with baselines for classification.

In [4]:
target = 'Great'
features = df.columns.drop([target] + ['Date'])

x_train = train[features]
x_validate = validate[features]
x_test = test[features]

y_train = train[target]
y_validate = validate[target]
y_test = test[target]

y_train.value_counts(normalize = True)

False    0.591216
True     0.408784
Name: Great, dtype: float64

## PART 4 --  Use scikit-learn for logistic regression.

In [0]:
# Used if viewing full df's in separate code cells
pd.options.display.max_columns = 999

In [0]:
# OneHotEncoding Features
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

x_train_encoded = encoder.fit_transform(x_train)
x_validate_encoded = encoder.fit_transform(x_validate)
x_test_encoded = encoder.fit_transform(x_test)

In [0]:
# Remove the nan columns because it's redundant to have BOTH nan columns (with
# 0's and 1's) and x columns (with 0's and 1's)
for column in dirty_columns:
  x_train_encoded = x_train_encoded.drop(column + '_nan', axis=1)
  x_validate_encoded = x_validate_encoded.drop(column + '_nan', axis=1)
  x_test_encoded = x_test_encoded.drop(column + '_nan', axis=1)

In [0]:
# I tried feature scaling, but I think it made my accuracy scores TOO accurate lol, I dont know...

# Feature scaling
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()

#x_train_encoded = scaler.fit_transform(x_train_encoded)
#x_validate_encoded = scaler.fit_transform(x_validate_encoded)
#x_test_encoded = scaler.fit_transform(x_test_encoded)

In [0]:
# SimplyImputing Features
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()

x_train_encoded_imputed = imputer.fit_transform(x_train_encoded)
x_validate_encoded_imputed = imputer.fit_transform(x_validate_encoded)
x_test_encoded_imputed = imputer.fit_transform(x_test_encoded)

In [10]:
# Logistic Regression to predict target
from sklearn.linear_model import LogisticRegression
log_r_train_model = LogisticRegression()
log_r_validate_model = LogisticRegression()
log_r_test_model = LogisticRegression()

log_r_train_model = log_r_train_model.fit(x_train_encoded_imputed, y_train)
log_r_validate_model = log_r_validate_model.fit(x_validate_encoded_imputed, y_validate)
log_r_test_model = log_r_test_model.fit(x_test_encoded_imputed, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## PART 5 -- Get your model's training, validation, and test accuracy. (Multiple times if you try multiple iterations.)

In [11]:
print('Validation Accuracy (Training Set)', log_r_train_model.score(x_train_encoded_imputed, y_train))
print('Validation Accuracy (Validation Set)', log_r_validate_model.score(x_validate_encoded_imputed, y_validate))
print('Validation Accuracy (Testing Set)', log_r_test_model.score(x_test_encoded_imputed, y_test))

Validation Accuracy (Training Set) 0.8986486486486487
Validation Accuracy (Validation Set) 0.9411764705882353
Validation Accuracy (Testing Set) 0.9210526315789473


# * Commit your notebook to your fork of the GitHub repo